In [1]:
# 1️⃣ Imports and Setup RESNET

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logging

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import ResNet50, InceptionV3
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_inception
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet


In [2]:
# Upload kaggle.json
from google.colab import files
files.upload()  # Upload kaggle.json here
# Setup Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# Download Dataset
!kaggle datasets download -d shubhamgoel27/dermnet
# Unzip Dataset
!unzip -q dermnet.zip -d dermnet
# Check files
import os
print(os.listdir("/content/dermnet"))
TRAIN_PATH = "/content/dermnet/train"
TEST_PATH = "/content/dermnet/test"


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/shubhamgoel27/dermnet
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
['test', 'train']


In [9]:
# Create an ImageDataGenerator with data augmentation RESNET
datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.3,
    rotation_range=30,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2  # Split training data into train and validation
)


In [10]:
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input as preprocess_effnet

IMAGE_SIZE = (299, 299)

# RESNET data generator
train_datagen_effnet = ImageDataGenerator(
    preprocessing_function=preprocess_effnet,
    shear_range=0.3,
    zoom_range=0.4,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

SEED = 42

train_generator = train_datagen_effnet.flow_from_directory(
    TRAIN_PATH,
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical',
    subset='training',
    seed=SEED
)

val_generator = train_datagen_effnet.flow_from_directory(
    TRAIN_PATH,
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    seed=SEED
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_effnet)

test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 12453 images belonging to 23 classes.
Found 3104 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.


In [11]:
effnet_base = EfficientNetV2S(include_top=False, weights='imagenet', input_shape=(299, 299, 3))
for layer in effnet_base.layers:
    layer.trainable = False  # Freeze for transfer learning RESNET

def build_effnet_model(base_model, num_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=predictions)

num_classes = train_generator.num_classes
effnet_model = build_effnet_model(effnet_base, num_classes)

effnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
y_train = train_generator.classes
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))


model train


In [13]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
]

effnet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=60,
    class_weight=class_weights,
    callbacks=callbacks
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/60
390/390 ━━━━━━━━━━━━━━━━━━━━ 503s 1s/step - accuracy: 0.1577 - loss: 2.9263 - val_accuracy: 0.2039 - val_loss: 2.7282 - learning_rate: 0.0010
Epoch 2/60
390/390 ━━━━━━━━━━━━━━━━━━━━ 416s 1s/step - accuracy: 0.2640 - loss: 2.5532 - val_accuracy: 0.1962 - val_loss: 2.7652 - learning_rate: 0.0010
Epoch 3/60
390/390 ━━━━━━━━━━━━━━━━━━━━ 417s 1s/step - accuracy: 0.2780 - loss: 2.4539 - val_accuracy: 0.2216 - val_loss: 2.7417 - learning_rate: 0.0010
Epoch 4/60
390/390 ━━━━━━━━━━━━━━━━━━━━ 418s 1s/step - accuracy: 0.2976 - loss: 2.3748 - val_accuracy: 0.2268 - val_loss: 2.7192 - learning_rate: 0.0010
Epoch 5/60
390/390 ━━━━━━━━━━━━━━━━━━━━ 479s 1s/step - accuracy: 0.2961 - loss: 2.3378 - val_accuracy: 0.2352 - val_loss: 2.7382 - learning_rate: 0.0010
Epoch 6/60
390/390 ━━━━━━━━━━━━━━━━━━━━ 431s 1s/step - accuracy: 0.3003 - loss: 2.3621 - val_accuracy: 0.2374 - val_loss: 2.7074 - learning_rate: 0.0010
Epoch 7/60
390/390 ━━━━━━━━━━━━━━━━━━━━ 418s 1s/step - accuracy: 0.3018 - loss: 2.

In [2]:
effnet_model.save("efficientnet_model.h5")


NameError: name 'effnet_model' is not defined

In [3]:
from tensorflow.keras.models import load_model
model = load_model("efficientnet_model.h5")

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'efficientnet_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [14]:

# 4️⃣ Compile and Callbacks. evaluation
effnet_loss, effnet_acc = effnet_model.evaluate(test_generator)
print(f'RESNET Test Accuracy: {effnet_acc:.4f}, Loss: {effnet_loss:.4f}')

y_true = test_generator.classes
y_pred = np.argmax(effnet_model.predict(test_generator), axis=1)

print("RESNET Classification Report:")
print(classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys())))


126/126 ━━━━━━━━━━━━━━━━━━━━ 35s 278ms/step - accuracy: 0.4029 - loss: 2.1242
EfficientNetV2 Test Accuracy: 0.3338, Loss: 2.2931
126/126 ━━━━━━━━━━━━━━━━━━━━ 46s 287ms/step
EfficientNetV2 Classification Report:
                                                                    precision    recall  f1-score   support

                                           Acne and Rosacea Photos       0.42      0.64      0.50       312
Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions       0.45      0.31      0.36       288
                                          Atopic Dermatitis Photos       0.27      0.40      0.32       123
                                            Bullous Disease Photos       0.17      0.14      0.16       113
                Cellulitis Impetigo and other Bacterial Infections       0.12      0.16      0.14        73
                                                     Eczema Photos       0.46      0.31      0.37       309
                                

In [17]:
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet

def predict(img):
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_resnet(img_array)  # ✅ FIXED!

    preds = model.predict(img_array)
    pred_index = np.argmax(preds)
    pred_class = CLASS_NAMES[pred_index]

    return pred_class


In [1]:
# ✅ Step 1: Install Gradio RESNET
!pip install -q gradio

# ✅ Step 2: Gradio app using your RESNET model
import gradio as gr
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input as preprocess_effnet
import numpy as np
from PIL import Image

# ✅ Use trained RESNET model
model = effnet_model  # Make sure you trained and defined this above

# ✅ Get class labels from the training generator (or test, whichever you have)
labels = list(train_generator.class_indices.keys())
CLASS_NAMES = labels

# ✅ Define prediction function
def predict(img):
    img = img.resize((299, 299))  # Input size for RESNET
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_effnet(img_array)  # Use RESNET preprocessing

    preds = model.predict(img_array)
    pred_index = np.argmax(preds)
    pred_class = CLASS_NAMES[pred_index]
    confidence = preds[0][pred_index]

    return f"{pred_class} ({confidence:.2%} confidence)"

# ✅ Launch Gradio Interface RESNET
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="🧠 Skin Disease Classifier by Tejasv Dua",
    description="Upload a skin image to detect disease using an model trained on 23 skin disease classes."
)

interface.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00


NameError: name 'effnet_model' is not defined

In [ ]:
from collections import Counter
print(Counter(train_generator_resnet.classes))



Counter({np.int32(14): 1124, np.int32(16): 1097, np.int32(18): 1040, np.int32(5): 988, np.int32(1): 920, np.int32(22): 869, np.int32(12): 832, np.int32(0): 672, np.int32(17): 485, np.int32(9): 455, np.int32(2): 392, np.int32(20): 386, np.int32(11): 371, np.int32(3): 359, np.int32(15): 345, np.int32(10): 336, np.int32(21): 333, np.int32(6): 324, np.int32(8): 324, np.int32(4): 231, np.int32(13): 208, np.int32(7): 192, np.int32(19): 170})
